In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
print(selenium.__version__)

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Read data from excel
# Read the Excel file
file_path = '../cities_scrap2.xlsx'  # Replace with your file path
sheet_name = 'Test'  # Replace with your sheet name
column_name_city = 'city'  # Replace with your column name
column_name_country = 'country'  # Replace with your column name

# Load the Excel file
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert the column to a list
city_list = df[column_name_city].tolist()
country_list = df[column_name_country].tolist()

# final list
city_names = []
for idx,city in enumerate(city_list):
    city_names.append(city_list[idx])
    city_names.append(str(city_list[idx])+'-'+str(country_list[idx]))

# city_names

In [ ]:
print(city_names)

In [3]:
# Initialize WebDriver

# Set up Chrome options to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up the WebDriver with the Chrome options
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()  # Assuming you're using Chrome

# Base URL
url_base = 'https://www.numbeo.com/quality-of-life/in/'

# Initialize an empty DataFrame
df_capital_all = pd.DataFrame(columns=[
    'index',
    'city',
    'climate_index',
    'cost_of_living_index',
    'health_care_index',
    'pollution_index',
    'property_price_to_income_ratio',
    'purchasing_power_index',
    'safety_index',
    'traffic_commute_time_index',
    'quality_of_life_index'
])

In [4]:
# Fetch data using Selenium

for idx,city in enumerate(city_names):

    try:

        url_city = url_base + city_names[idx]
        driver.get(url_city)

        # Find Table element
        tbl_quality = driver.find_element(By.XPATH, '/html/body/div[2]/table')  # if there's no ID, always a good idea to copy xpath

        
        # Parse Table element into pandas DataFrame
        tbl_quality_html = tbl_quality.get_attribute('outerHTML')
        soup = BeautifulSoup(tbl_quality_html, 'html.parser')

        # Find all table rows in the parsed HTML
        tbl_quality_rows = soup.find_all('tr')

        # Initialize data list
        data_cost = []
        for row in tbl_quality_rows:
                row_data = []
                cells = row.find_all(['td', 'th'])
                for cell in cells:
                    row_data.append(cell.get_text().strip())
                data_cost.append(row_data)

        # Extract header and data rows separately
        header = ['kpi_name','kpi_value','kpi_hml']

        # Create DataFrame
        df_capital = pd.DataFrame(data_cost, columns=header)

        # Clean data
        df_capital['Index'] = idx
        df_capital['City'] = city_names[idx]
        df_capital.drop(columns=['kpi_hml'], inplace=True)
        df_capital['kpi_value'] = pd.to_numeric(df_capital['kpi_value'], errors='coerce')
        # df_capital = df_capital[~df_capital['kpi_name'].isna()].copy()
        df_capital.drop(8, inplace=True)


        # Transpose df
        df_capital_t = df_capital.pivot_table(index=['Index','City'], columns='kpi_name', values='kpi_value', dropna=False)
        df_capital_t.reset_index(inplace=True)
        df_capital_t.columns = df_capital_all.columns

        # Append data to complete DF
        df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)

    except:
         print(city)

Charikar
Charikar-Afghanistan
Farah
Farah-Afghanistan
Jalalabad
Jalalabad-Afghanistan


C:\Users\paude\AppData\Local\Temp\ipykernel_33308\3266304027.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_capital_all = pd.concat([df_capital_all,df_capital_t], axis=0)


Kabul-Afghanistan
Kandahar
Kandahar-Afghanistan
Mazar-E-Sharif
Mazar-E-Sharif-Afghanistan
Annaba
Batna
Batna-Algeria
Blida
Blida-Algeria
Chlef
Chlef-Algeria
El-Khroub
El-Khroub-Algeria
Khemis-Miliana
Khemis-Miliana-Algeria
Mila
Mila-Algeria
M%27Sila
M%27Sila-Algeria
Benguela
Benguela-Angola
Malanje
Malanje-Angola
Saurimo
Saurimo-Angola
Saint-John%27S
Saint-John%27S-Antigua-And-Barbuda
Ayacucho
Ayacucho-Argentina
Bahia-Blanca
Buenos-Aires-Argentina
Clorinda
Clorinda-Argentina
Comodoro-Rivadavia
Comodoro-Rivadavia-Argentina
Concordia
Concordia-Argentina
Ensenada-Argentina
Formosa
Formosa-Argentina
La-Paz-Argentina
La-Paz-Argentina
La-Plata
Mendoza
Mercedes
Mercedes-Argentina
Mercedes
Mercedes-Argentina
Rosario-Argentina
San-Juan-Argentina
San-Salvador-De-Jujuy
San-Salvador-De-Jujuy-Argentina
Santa-Rosa-Argentina
Santa-Rosa-Argentina
Santiago-Del-Estero
Santiago-Del-Estero-Argentina
Victoria-Argentina
Villa-Mercedes
Villa-Mercedes-Argentina
Armavir
Armavir-Armenia
Adelaide-Australia
Alice

In [5]:
df_capital_all.head(20)
# df_capital_all.shape

,index,city,climate_index,cost_of_living_index,health_care_index,pollution_index,property_price_to_income_ratio,purchasing_power_index,safety_index,traffic_commute_time_index,quality_of_life_index
0,6,Kabul,NaN,NaN,26.16,89.71,9.61,NaN,21.05,56.17,NaN
0,13,Annaba-Algeria,97.28,NaN,76.85,95.69,12.93,NaN,36.56,15.00,NaN
0,39,Bahia-Blanca-Argentina,91.54,NaN,67.59,63.45,20.00,NaN,54.15,30.00,NaN
0,40,Buenos-Aires,98.28,32.62,68.39,51.76,22.89,32.80,36.54,49.79,105.95
0,48,Ensenada,96.82,NaN,93.60,43.83,NaN,NaN,60.68,37.00,NaN
0,52,La-Paz,55.46,30.03,47.01,75.36,12.06,40.12,39.56,52.67,81.49
0,54,La-Paz,55.46,30.03,47.01,75.36,12.06,40.12,39.56,52.67,81.49
0,57,La-Plata-Argentina,97.52,NaN,60.19,60.20,7.00,NaN,31.53,39.11,NaN
0,59,Mendoza-Argentina,87.76,26.28,58.33,36.18,11.61,50.49,34.42,36.50,133.39
0,64,Rosario,96.13,27.81,75.53,42.97,12.70,52.00,24.44,25.61,138.34


In [6]:
df_capital_all.shape

(582, 11)

In [7]:
df_capital_all.to_csv('test_out_final_2.csv')

In [8]:
# # Define the file path
# file_path = r'C:\Users\paude\Documents\00. Backup 2023\EXTERIOR\MASTER\MA IAAC\03. Cursada\01. Modulo 03\S01 DE\Class Project\DE_Team\Scraped_csvs\data_cost.csv'

# # Export DataFrame to a CSV file
# df_capital_2.to_csv(file_path, index=False)

# print("DataFrame exported successfully to:", file_path)